In [487]:
include("SchwarzChristoffel.jl")

SchwarzChristoffel

In [488]:
p = SchwarzChristoffel.Polygon([-1.0,1.0,0.5,-1.0],[-1.0,-1.0,1.0,1.0])

Polygon with 4 vertices at Complex{Float64}[-1.0-1.0im, 1.0-1.0im, 0.5+1.0im, -1.0+1.0im] 
             interior angles/π = [0.5, 0.422, 0.578, 0.5]


In [ ]:
SchwarzChristoffel.Polygons.plot(p)

In [ ]:
z = [0.71+im*0.2 0.2+im*0]
SchwarzChristoffel.Polygons.plot(p)
PyPlot.plot(real.(z),imag.(z),marker="o")
SchwarzChristoffel.Polygons.isinpoly.(z,p,1e-5)

In [464]:
z1 = -1.0+0.0im #1.0im
z2 = exp(im*5π/4) #exp(im*3π/4)
sing1 = 2 #1
beta = [0.5,0.5,0.5,0.5]
z = [1.0im,-1.0,-1.0im,1.0]
nqpts = 8
qdat = SchwarzChristoffel.Integration.qdata(beta,nqpts)
dquad = SchwarzChristoffel.Exterior.DQuad(z,beta,qdat)
dabsquad2 = SchwarzChristoffel.Exterior.DabsQuad(z,beta,qdat)
I = dabsquad2(z1,z2,sing1)

[1.43179 0.0250195 1.39641 1.99984; 1.48104 0.0968518 1.34407 1.99765; 1.5524 0.206077 1.26097 1.98935; 1.63292 0.338078 1.1548 1.97122; 1.70968 0.475107 1.03778 1.94275; 1.77295 0.599196 0.925556 1.90813; 1.81753 0.695022 0.83462 1.87535; 1.84207 0.751734 0.778959 1.85335]


0.8472130847939816

In [454]:
wt = Complex{Float64}[0.00288252-0.00695902im, 0.0106214-0.0256423im, 0.0207508-0.0500968im, 0.0299546-0.0723169im, 0.0350199-0.0845455im, 0.033766-0.0815182im, 0.0256871-0.0620141im, 0.0121433-0.0293165im]

8-element Array{Complex{Float64},1}:
 0.00288252-0.00695902im
  0.0106214-0.0256423im 
  0.0207508-0.0500968im 
  0.0299546-0.0723169im 
  0.0350199-0.0845455im 
   0.033766-0.0815182im 
  0.0256871-0.0620141im 
  0.0121433-0.0293165im 

In [455]:
expterm = Complex{Float64}[1.64901-1.17127im, 1.58504-1.34517im, 1.41529-1.5949im, 1.10214-1.82286im, 0.70388-1.91522im, 0.352431-1.8556im, 0.127754-1.7334im, 0.0221179-1.64037im]

8-element Array{Complex{Float64},1}:
   1.64901-1.17127im
   1.58504-1.34517im
   1.41529-1.5949im 
   1.10214-1.82286im
   0.70388-1.91522im
  0.352431-1.8556im 
  0.127754-1.7334im 
 0.0221179-1.64037im

In [461]:
transpose(wt)*expterm

-0.59906925001173 - 0.59906962658455im

In [416]:
z1 = [-1.0+0.0im,0.0+1.0im]
z2 = [exp(im*5π/4),exp(im*3π/4)]
sing1 = [2,1]
I = dquad.(z1,z2,sing1)

2-element Array{Complex{Float64},1}:
 -0.169443-0.565685im
  0.565685-0.169443im

In [338]:
function deparam(w::Vector{Complex128},beta::Vector{Float64},
                 zeta0::Vector{Complex128},qdat)
  # w clockwise
  # beta turning angles

  n = length(w)
  if n == 2
    zeta = [-1,1]
  else
    len = abs.(diff(circshift(w,1)))
    nmlen = abs.(len[3:n-1]/len[2])
    if isempty(zeta0)
      y0 = zeros(n-1)
    else
      zeta0 = zeta0/zeta0[n]
      θ = angle.(zeta0)
      θ[θ.<=0] = θ[θ.<=0] + 2π
      dt = diff([0;θ[1:n-1];2π])
      @. y0 = log(dt[1:n-1]/dt[2:n])
    end

    depfun!(F,y) = SchwarzChristoffel.Exterior.depfunfull!(F,y,n,beta,nmlen,qdat)

    F0 = similar(y0)
    df = OnceDifferentiable(depfun!, y0, F0)
    sol = nlsolve(depfun!,y0,show_trace = :true)

    zeta, θ = y_to_zeta(sol.zero)

  end

  mid = zeta[1]*exp(0.5*im*angle(zeta[2]/zeta[1]))
  dabsquad = SchwarzChristoffel.Exterior.DabsQuad(zeta,beta,qdat)
  c = (w[2]-w[1])/(dabsquad(zeta[1],mid,1)-dabsquad(zeta[2],mid,2))

  return zeta, c

end
function y_to_zeta(y::Vector{Float64})

    cs = cumsum(cumprod([1;exp.(-y)]))
    n = length(cs)
    θ = 2π*cs[1:n-1]./cs[n]
    zeta = ones(Complex128,n)
    zeta[1:n-1] = exp.(im*θ)
    return zeta, θ

  end

y_to_zeta (generic function with 1 method)

In [485]:
function depfunfull!(F,y,n,beta,nmlen,qdat)  
    
  zeta, θ = SchwarzChristoffel.Exterior.y_to_zeta(y)  
  mid = exp.(im*0.5*(θ[1:n-2]+θ[2:n-1]))
  #dabsquad(z1,z2,sing1) = SchwarzChristoffel.Exterior.dabsquad(z1,z2,sing1,zeta,beta,qdat)
  dabsquad = SchwarzChristoffel.Exterior.DabsQuad(zeta,beta,qdat)
  ints = dabsquad.(zeta[1:n-2],mid,collect(1:n-2)) + dabsquad.(zeta[2:n-1],mid,collect(2:n-1))
  
    println(qdat[1])
    println(qdat[2])    
  if n > 3
    F[1:n-3] = abs.(ints[2:n-2])/abs(ints[1]) - nmlen
  end

  res = -sum(beta./zeta)/ints[1]
  F[n-2] = real(res)
  F[n-1] = imag(res)  
    
end

depfunfull! (generic function with 2 methods)

In [486]:
w = flipdim(p.vert,1)
beta = 1.0-flipdim(p.angle,1)
len = abs.(diff(circshift(w,1)))
nmlen = abs.(len[3:n-1]/len[2])
F = zeros(n-1)
y0 = zeros(n-1)
qdat = 
depfunfull!(F,y0,n,beta,nmlen,qdat)
println(F)
SchwarzChristoffel.Exterior.depfunfull!(F,y0,n,beta,nmlen,qdat)
println(F)

[-0.936287 -0.936287 -0.936287 -0.936287 -0.96029; -0.753272 -0.753272 -0.753272 -0.753272 -0.796666; -0.474297 -0.474297 -0.474297 -0.474297 -0.525532; -0.134937 -0.134937 -0.134937 -0.134937 -0.183435; 0.221528 0.221528 0.221528 0.221528 0.183435; 0.549642 0.549642 0.549642 0.549642 0.525532; 0.807568 0.807568 0.807568 0.807568 0.796666; 0.96248 0.96248 0.96248 0.96248 0.96029]
[0.031818 0.031818 0.031818 0.031818 0.101229; 0.117242 0.117242 0.117242 0.117242 0.222381; 0.229053 0.229053 0.229053 0.229053 0.313707; 0.330647 0.330647 0.330647 0.330647 0.362684; 0.386559 0.386559 0.386559 0.386559 0.362684; 0.372718 0.372718 0.372718 0.372718 0.313707; 0.283541 0.283541 0.283541 0.283541 0.222381; 0.134041 0.134041 0.134041 0.134041 0.101229]
[-0.45049, -0.0423905, -0.0423905]
[-0.45049, -0.0423905, -0.0423905]


In [ ]:
w = flipdim(p.vert,1)
beta = 1.0-flipdim(p.angle,1)
zeta0 = Complex128[]
n = length(w)
tol = 1e-8
nqpts = max(ceil(Int,-log10(tol)),2)
qdat = SchwarzChristoffel.Integration.qdata(beta,nqpts)
#len = abs.(diff(circshift(w,1)))
#nmlen = abs.(len[3:n-1]/len[2])
#depfun!(F,y) = depfunfull!(F,y,n,beta,nmlen,qdat)

#y0 = zeros(n-1)
#F0 = similar(y0)
#df = OnceDifferentiable(depfun!, y0, F0)
#sol = nlsolve(depfun!,y0,show_trace = :true)

#zeta, θ = SchwarzChristoffel.Exterior.y_to_zeta(sol.zero)
#zeta
zeta, c = SchwarzChristoffel.Exterior.deparam(w,beta,zeta0,qdat)

In [491]:
zeta

4-element Array{Complex{Float64},1}:
 -0.0195313+0.999809im
  -0.984827+0.173541im
    -0.1291-0.991632im
        1.0+0.0im     

In [489]:
SchwarzChristoffel.Exterior.Extermap(p)

LoadError: [91mMethodError: no method matching deparam(::Array{Complex{Float64},1}, ::Array{Float64,1}, ::Array{Complex{Float64},1})[0m
Closest candidates are:
  deparam(::Array{Complex{Float64},1}, ::Array{Float64,1}, ::Array{Complex{Float64},1}, [91m::Any[39m) at /Users/jeff/.julia/v0.6/SchwarzChristoffel/src/Exterior.jl:175[39m

In [ ]:
deparam(flipdim(p.vert,1),1.-flipdim(p.angle,1),Complex128[],qdat)

In [419]:
zeta,c = SchwarzChristoffel.Exterior.deparam(w,beta,zeta0,qdat)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     3.743685e-01              NaN
     1     7.992707e-02     3.772555e-01
     2     2.169546e-03     5.531535e-02
     3     1.718285e-06     1.586825e-03
     4     1.080469e-12     1.258762e-06


(Complex{Float64}[-0.144345+0.989527im, -1.0+5.66554e-16im, 0.144345-0.989527im, 1.0+0.0im], 1.0273217604172857 + 1.1880523385366173im)

In [421]:
zeta

4-element Array{Complex{Float64},1}:
 -0.144345+0.989527im   
      -1.0+5.66554e-16im
  0.144345-0.989527im   
       1.0+0.0im        

In [90]:
θ = angle.(p.vert)
θ[θ.<=0] = θ[θ.<=0] + 2π
diff([0;θ[1:end-1];2π])

4-element Array{Float64,1}:
  3.92699
  1.5708 
 -4.39064
  5.17604

In [37]:
function arraystuff2(a::Vector{Float64},b::Vector{Complex128})
    @assert length(a)==length(b)
    a[1]
end

arraystuff2 (generic function with 1 method)

In [41]:
arraystuff2([1.0;2.0],Complex128[1;4;5])

LoadError: [91mAssertionError: length(a) == length(b)[39m

In [45]:
typeof(Complex128[])

Array{Complex{Float64},1}

In [53]:
typeof(max(ceil(Int,-log10(1e-8)),2))

Int64

In [49]:
sizeof(Integer)

LoadError: [91margument is an abstract type; size is indeterminate[39m